## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()

['eigen-vecs', 'stations']


latitude  longitude  elevation state                   name  \
ACW00011604   17.1167   -61.7833       10.1   NaN  ST JOHNS COOLIDGE FLD   
ACW00011647   17.1333   -61.7833       19.2   NaN               ST JOHNS   
AE000041196   25.3330    55.5170       34.0   NaN    SHARJAH INTER. AIRP   
AF000040930   35.3170    69.0170     3366.0   NaN           NORTH-SALANG   
AG000060390   36.7167     3.2500       24.0   NaN     ALGER-DAR EL BEIDA   

            GSNFLAG HCNFLAG    WMOID  
ACW00011604     NaN     NaN      NaN  
ACW00011647     NaN     NaN      NaN  
AE000041196     GSN     NaN  41196.0  
AF000040930     GSN     NaN  40930.0  
AG000060390     GSN     NaN  60390.0

In [4]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
# Eig.head()
Eig

0         1         2         3
0   -0.149073  0.090741 -0.034700  0.064624
1   -0.108335  0.133973 -0.040574 -0.007475
2   -0.138108  0.026584 -0.022607  0.095156
3   -0.141599  0.078494 -0.022098  0.088553
4   -0.144295  0.025554 -0.019818  0.113470
5   -0.136168  0.031129 -0.025119  0.086857
6   -0.131825  0.037797 -0.034485  0.055047
7   -0.140364  0.023702 -0.016193  0.116053
8   -0.151649  0.092653 -0.035295  0.062849
9   -0.122674  0.042781 -0.037285  0.031026
10  -0.148142  0.101374 -0.035637  0.057538
11  -0.086361  0.122611 -0.053936 -0.071358
12  -0.168235  0.088080 -0.028431  0.083445
13  -0.168848  0.091640 -0.026690  0.094722
14  -0.155855  0.091496 -0.028537  0.067939
15  -0.164382  0.089006 -0.026571  0.084826
16  -0.132939  0.024722 -0.014761  0.096598
17  -0.148173  0.099104 -0.044936  0.037096
18  -0.132087  0.103363 -0.044082  0.017260
19  -0.149949  0.108933 -0.047347  0.030898
20  -0.107447  0.027810 -0.024560  0.041290
21  -0.127372  0.036452 -0.020939  0.099648
22  -0.060985  0.107865 -0.108670 -0.183044
23  -0.135016  0.062394 -0.013906  0.102767
24  -0.134465  0.078674 -0.018234  0.077192
25  -0.138248  0.081511 -0.025411  0.075879
26  -0.142933  0.074960 -0.020117  0.093120
27  -0.135616  0.067035 -0.015788  0.088063
28  -0.128064  0.018855 -0.011288  0.101483
29  -0.103511  0.052697 -0.021065  0.037146
..        ...       ...       ...       ...
87   0.057338  0.046482 -0.135948 -0.118626
88   0.109223  0.071425 -0.049012  0.073388
89   0.043417  0.061683 -0.060489 -0.122617
90   0.064332  0.016409 -0.207510  0.061757
91   0.067810  0.021247 -0.203162  0.078431
92   0.031344  0.040186 -0.052554  0.050191
93  -0.000736  0.088108 -0.040174 -0.177698
94   0.074623  0.060854  0.051015 -0.033442
95   0.066223  0.113201  0.007408  0.052537
96   0.061264  0.017823 -0.178851  0.072938
97   0.044647  0.076467  0.080340  0.007067
98  -0.005817  0.117377  0.060552 -0.021542
99   0.013932  0.058247 -0.001017 -0.041229
100 -0.001776  0.079281 -0.048525 -0.186569
101  0.092904  0.045163 -0.000735  0.076135
102  0.061618  0.132219  0.035025  0.007586
103  0.101184  0.091496  0.027336  0.072397
104  0.072530  0.011710 -0.235442  0.072555
105  0.089140  0.169759  0.113747  0.047961
106  0.096489  0.043544 -0.112944  0.061335
107  0.030138  0.016199 -0.129517 -0.076006
108  0.022694  0.026363 -0.124379 -0.123552
109  0.022756  0.022512 -0.095407 -0.091762
110  0.094823  0.072540 -0.118946 -0.004207
111 -0.068282  0.110318 -0.094759 -0.226106
112  0.010866  0.040889 -0.057637 -0.079610
113  0.030879  0.017754 -0.128977 -0.129288
114 -0.068933  0.114159 -0.093080 -0.220343
115 -0.064235  0.104658 -0.092101 -0.223425
116 -0.067283  0.118133 -0.101943 -0.238763

[117 rows x 4 columns]

In [5]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00045995', u'USC00044823', u'USC00044633', u'US1CASM0014', u'USC00046201']


0         1         2         3
station                                            
USC00045995 -0.149073  0.090741 -0.034700  0.064624
USC00044823 -0.108335  0.133973 -0.040574 -0.007475
USC00044633 -0.138108  0.026584 -0.022607  0.095156
US1CASM0014 -0.141599  0.078494 -0.022098  0.088553
USC00046201 -0.144295  0.025554 -0.019818  0.113470

In [6]:
table=Eig.join(Stations,how='left')

In [7]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]] # ,4,5,6,7]]
# table.head(4)
table

name  latitude  longitude  elevation  \
station                                                                  
USC00045995           MT TAMALPAIS 1 S   37.9167  -122.5667      289.6   
USC00044823     LATHROP SAN JOAQUIN BD   37.7833  -121.3000        8.2   
USC00044633           LA FAYETTE 2 NNE   37.9167  -122.1000      164.9   
US1CASM0014            BELMONT 0.7 WSW   37.5104  -122.3057      146.9   
USC00046201                      NILES   37.5833  -121.9667       21.3   
USC00048376               S E FARALLON   37.7000  -123.0000       14.0   
USC00045915         MT DIABLO JUNCTION   37.8792  -121.9303      661.4   
USC00045032                       LODI   38.1061  -121.2878       12.2   
USC00049001          TRACY PUMPING PLT   37.7967  -121.5828       18.6   
USC00044508                  KERLINGER   37.6833  -121.4333       53.9   
USC00044500                  KENTFIELD   37.9567  -122.5447       44.2   
US1CASM0001             PACIFICA 0.3 W   37.6121  -122.4825       69.2   
US1CASM0006         FOSTER CITY 1.3 SW   37.5488  -122.2587        2.1   
US1CASM0008        REDWOOD CITY 2.5 SW   37.4931  -122.2399        5.2   
USC00046332                    OAKLAND   37.7833  -122.1667      134.1   
USC00046333          OAKLAND CITY HALL   37.8000  -122.2667       10.7   
USC00046336             OAKLAND MUSEUM   37.7983  -122.2642        9.1   
USW00023272         SAN FRANCISCO DWTN   37.7706  -122.4269       45.7   
USC00045296          MANDEVILLE ISLAND   38.0333  -121.5667        3.0   
USC00044997                  LIVERMORE   37.6922  -121.7692      146.3   
USC00043650      GRIZZLY ISLAND REFUGE   38.1500  -121.9667     -999.9   
USC00043652               GRIZZLY PEAK   37.8667  -122.2167      534.0   
USC00045378               MARTINEZ WTP   38.0131  -122.1142       12.2   
USC00042177                   CROCKETT   38.0333  -122.2167        3.0   
USC00045377          MARTINEZ FIRE STN   38.0167  -122.1333        9.1   
USC00045371               MARTINEZ 2 S   37.9700  -122.1275       70.1   
USC00045372             MARTINEZ 3 SSE   37.9667  -122.1000       78.9   
USC00049420              WALMAR SCHOOL   37.9500  -122.0833       36.9   
USC00049423         WALNUT CREEK 2 ESE   37.8833  -122.0333       74.7   
USC00049427           WALNUT CREEK 4 E   37.9000  -121.9833      121.9   
...                                ...       ...        ...        ...   
US1CAAL0012         PLEASANTON 0.4 ESE   37.6702  -121.8814      104.9   
US1CAAL0011        SAN LORENZO 0.5 NNW   37.6821  -122.1332        9.1   
USC00049185  UPPER SAN LEANDRO FILTERS   37.7719  -122.1675      120.1   
US1CACC0018       WALNUT CREEK 1.4 SSE   37.8846  -122.0285       84.4   
US1CACC0016            CONCORD 0.8 WNW   37.9783  -122.0134       37.8   
US1CASF0004       SAN FRANCISCO 1.1 SW   37.7650  -122.4348       59.1   
US1CACC0011            MARTINEZ 2.2 SW   37.9744  -122.1384      210.9   
US1CACC0010          LAFAYETTE 1.9 WSW   37.8883  -122.1526      157.9   
USC00041206                 BURLINGAME   37.5833  -122.3500        3.0   
USC00041043             BRANNAN ISLAND   38.1167  -121.7000        9.1   
USC00046598                PACIFICA 2S   37.6103  -122.4842       38.1   
USC00046599             PACIFICA 4 SSE   37.5917  -122.4717      144.8   
US1CACV0002             WALLACE 0.3 NE   38.1967  -120.9728       90.8   
US1CAMR0012          SAN RAFAEL 3.9 NW   38.0268  -122.5473       21.9   
US1CAMR0013          MILL VALLEY 1.5 S   37.8863  -122.5440       63.7   
USC00048559             STOCKTON 5 SSE   37.8992  -121.2514        6.7   
USC00048557                 STOCKTON 2   37.9500  -121.2667        6.1   
USC00048554      STOCKTON DISPOSAL PLT   37.9333  -121.3333        3.0   
USW00023211           HAMILTON AF BASE   38.0667  -122.5167        4.0   
USC00040693                   BERKELEY   37.8744  -122.2606       94.5   
US1CACC0008           CROCKETT 0.7 WNW   38.0545  -122.2327       64.9   
USC00041583        CASTLE ROCK RAD LAB

In [8]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [9]:
# min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)
min_lat,max_lat,min_long,max_long = box = (37.6242, 38.2, -122.2606, -120.8667)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.